In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import xarray as xr

import pystac_client
import planetary_computer as pc

from datetime import date, timedelta
from tqdm import tqdm
import os

In [ ]:
def load_terraclimate_dataset():
    """Open the full TerraClimate Zarr store from Planetary Computer."""
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )
    collection = catalog.get_collection("terraclimate")
    asset = collection.assets["zarr-abfs"]

    if "xarray:storage_options" in asset.extra_fields:
        ds = xr.open_zarr(
            asset.href,
            storage_options=asset.extra_fields["xarray:storage_options"],
            consolidated=True,
        )
    else:
        ds = xr.open_dataset(
            asset.href,
            **asset.extra_fields["xarray:open_kwargs"],
        )

    return ds


def extract_terraclimate(row, ds, spatial_buffer=0.125, months_back=12):
    """
    Extract a TerraClimate spatiotemporal patch for a single sample.

    Args:
        row: DataFrame row with Latitude, Longitude, Sample Date
        ds: Full TerraClimate xr.Dataset (opened once)
        spatial_buffer: Degrees around the point (~0.042 deg per pixel at ~4km)
                        Default 0.125 gives roughly a 7x7 pixel patch
        months_back: Number of months before the sample date to include.
                     Default 12 captures a full seasonal cycle.

    Returns:
        xr.Dataset with dims (time, lat, lon) and all TerraClimate variables, or None.
    """
    lat = row['Latitude']
    lon = row['Longitude']
    sample_date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

    if pd.isna(sample_date):
        return None

    # Temporal window: months_back months before sample date up to sample date
    end_date = sample_date
    start_date = sample_date - pd.DateOffset(months=months_back)

    # Spatial window
    lat_min, lat_max = lat - spatial_buffer, lat + spatial_buffer
    lon_min, lon_max = lon - spatial_buffer, lon + spatial_buffer

    try:
        patch = ds.sel(
            time=slice(str(start_date.date()), str(end_date.date())),
            lat=slice(lat_max, lat_min),   # lat is descending in TerraClimate
            lon=slice(lon_min, lon_max),
        )

        if patch.dims['time'] == 0 or patch.dims['lat'] == 0 or patch.dims['lon'] == 0:
            return None

        # Load into memory (small slice)
        patch = patch.load()

        # Attach metadata
        patch.attrs['sample_lat'] = lat
        patch.attrs['sample_lon'] = lon
        patch.attrs['sample_date'] = str(sample_date.date())

        return patch

    except Exception as e:
        print(f"  Error for ({lat}, {lon}): {e}")
        return None


def save_batch(patches, metadata, batch_idx, output_dir, target_spatial=7, target_time=13):
    """
    Combine patches into a single .nc file.

    Pads/crops spatial dims to (target_spatial, target_spatial)
    and time dim to target_time so all batches are uniform.

    Saved Dataset has dims (sample, time, lat, lon).
    """
    if not patches:
        return

    band_names = [v for v in patches[0].data_vars]

    # Pad each patch to uniform dims
    padded_bands = {band: [] for band in band_names}

    for p in patches:
        for band in band_names:
            arr = p[band].values.astype("float32")
            t, h, w = arr.shape

            out = np.full((target_time, target_spatial, target_spatial), np.nan, dtype="float32")
            out[:min(t, target_time), :min(h, target_spatial), :min(w, target_spatial)] = \
                arr[:min(t, target_time), :min(h, target_spatial), :min(w, target_spatial)]
            padded_bands[band].append(out)

    # Stack into (n_samples, time, lat, lon) per band
    batch_ds = xr.Dataset({
        band: (["sample", "time", "y", "x"], np.stack(padded_bands[band]))
        for band in band_names
    })

    # Attach metadata as coordinates
    batch_ds.coords["lat"] = ("sample", [m["lat"] for m in metadata])
    batch_ds.coords["lon"] = ("sample", [m["lon"] for m in metadata])
    batch_ds.coords["sample_date"] = ("sample", [m["sample_date"] for m in metadata])
    batch_ds.coords["original_index"] = ("sample", [m["original_index"] for m in metadata])

    path = os.path.join(output_dir, f"batch_{batch_idx:03d}.nc")
    batch_ds.to_netcdf(path)
    print(f"  Saved {path} ({len(patches)} patches)")

In [ ]:
DATA_DIR = os.path.join("..", "..", "Data")
BATCH_SIZE = 100
SPATIAL_BUFFER = 0.125   # ~7 pixels at ~4km
MONTHS_BACK = 12         # full seasonal cycle

train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
val_df = pd.read_csv(os.path.join(DATA_DIR, "validation.csv"))

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# Open TerraClimate once (lazy Zarr, no download yet)
ds = load_terraclimate_dataset()
print(f"\nTerraClimate variables ({len(ds.data_vars)}): {list(ds.data_vars)}")
print(f"Time range: {str(ds.time.values[0])[:10]} to {str(ds.time.values[-1])[:10]}")
print(f"Spatial resolution: ~{abs(float(ds.lat[1] - ds.lat[0])):.4f} deg")

### Extract Training Patches
Processes in batches of 100. Each batch is saved as a `.nc` file immediately, so progress is preserved if interrupted. Already-saved batches are skipped on re-run.

In [ ]:
train_output_dir = os.path.join(DATA_DIR, "terraclimate_patches", "train")
os.makedirs(train_output_dir, exist_ok=True)

n_batches = (len(train_df) + BATCH_SIZE - 1) // BATCH_SIZE
failed_indices = []

for batch_idx in range(n_batches):
    batch_path = os.path.join(train_output_dir, f"batch_{batch_idx:03d}.nc")

    if os.path.exists(batch_path):
        print(f"Batch {batch_idx:03d} already exists, skipping.")
        continue

    start = batch_idx * BATCH_SIZE
    end = min(start + BATCH_SIZE, len(train_df))
    batch_df = train_df.iloc[start:end]

    patches = []
    metadata = []

    print(f"Batch {batch_idx:03d} ({start}-{end-1}):")
    for idx, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc="  Extracting"):
        patch = extract_terraclimate(row, ds, spatial_buffer=SPATIAL_BUFFER, months_back=MONTHS_BACK)
        if patch is not None:
            patches.append(patch)
            metadata.append({
                "lat": patch.attrs["sample_lat"],
                "lon": patch.attrs["sample_lon"],
                "sample_date": patch.attrs["sample_date"],
                "original_index": idx,
            })
        else:
            failed_indices.append(idx)

    save_batch(patches, metadata, batch_idx, train_output_dir)

print(f"\nDone. {len(failed_indices)} failed samples: {failed_indices[:20]}{'...' if len(failed_indices) > 20 else ''}")

### Extract Validation Patches

In [ ]:
val_output_dir = os.path.join(DATA_DIR, "terraclimate_patches", "validation")
os.makedirs(val_output_dir, exist_ok=True)

n_batches_val = (len(val_df) + BATCH_SIZE - 1) // BATCH_SIZE
failed_indices_val = []

for batch_idx in range(n_batches_val):
    batch_path = os.path.join(val_output_dir, f"batch_{batch_idx:03d}.nc")

    if os.path.exists(batch_path):
        print(f"Batch {batch_idx:03d} already exists, skipping.")
        continue

    start = batch_idx * BATCH_SIZE
    end = min(start + BATCH_SIZE, len(val_df))
    batch_df = val_df.iloc[start:end]

    patches = []
    metadata = []

    print(f"Batch {batch_idx:03d} ({start}-{end-1}):")
    for idx, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc="  Extracting"):
        patch = extract_terraclimate(row, ds, spatial_buffer=SPATIAL_BUFFER, months_back=MONTHS_BACK)
        if patch is not None:
            patches.append(patch)
            metadata.append({
                "lat": patch.attrs["sample_lat"],
                "lon": patch.attrs["sample_lon"],
                "sample_date": patch.attrs["sample_date"],
                "original_index": idx,
            })
        else:
            failed_indices_val.append(idx)

    save_batch(patches, metadata, batch_idx, val_output_dir)

print(f"\nDone. {len(failed_indices_val)} failed samples: {failed_indices_val[:20]}{'...' if len(failed_indices_val) > 20 else ''}")

### Verify Saved Batches

In [ ]:
sample_batch = xr.open_dataset(os.path.join(train_output_dir, "batch_000.nc"))
print("Dims:", dict(sample_batch.dims))
print("Variables:", list(sample_batch.data_vars))
print("Coords:", list(sample_batch.coords))
print(f"\nSample 0: lat={float(sample_batch.lat[0])}, lon={float(sample_batch.lon[0])}, date={str(sample_batch.sample_date[0].values)}")
sample_batch